In [1]:
import pandas as pd
import matplotlib.pyplot as plt
from ipywidgets import widgets
from ipywidgets import interactive
from IPython.display import Audio, display
import numpy as np
# from ipywidgets import widgets as w
# from bqplot import LinearScale, DateScale, Axis, Lines, Figure, Tooltip, Scatter
# import bqplot
# from bqplot import pyplot as bqplt

In [2]:
from colormath.color_objects import XYZColor, sRGBColor
from colormath.color_conversions import convert_color
from colormath.color_objects import SpectralColor

In [3]:
def get_spec_for_cm(wvl, amp):
    """Get a pandas series with appropriate index"""
    # 
    assert 340 <= wvl <= 830, "we require 340 < wvl < 830"
    spec = pd.Series(index=range(340, 831, 10), data=0.0)
    spec[int(np.round(wvl/10.)*10.)] = amp  # round the signals to the nearest 5.0
    return spec

# do summation on this pandas series

def get_cm_sc_args(spec):
    spec_dict = spec[spec>0].to_dict()
    return {f"spec_{wvl}nm": spec_dict[wvl] for wvl in spec_dict}

def cmsc_from_spec(spec):
    return SpectralColor(**get_cm_sc_args(spec))

In [8]:
from matplotlib_venn import venn3, venn3_circles
from itertools import product


def tritone_metamers_cm(wvl_1=460, wvl_2=530, wvl_3=610.60, a1=0.75, a2=0.85, a3=0.85):
    """autoplay sound and matplotlib display"""
    max_time = 15 # make sound file short to speed loading for interaction
    rate = 8000
    
    amp_tot = a1 + a2 + a3
    if amp_tot > 1.0:
        a1/=amp_tot
        a2/=amp_tot
        a3/=amp_tot
    pitch_1 = 1000 - wvl_1  # want to make the pitch change faster
    pitch_2 = 1000 - wvl_2
    pitch_3 = 1000 - wvl_3
    times = np.linspace(0,max_time,rate*max_time)
    sig1 = a1*np.sin(2*np.pi*pitch_1*times) 
    sig2 = a2*np.sin(2*np.pi*pitch_2*times)
    sig3 = a3*np.sin(2*np.pi*pitch_3*times)
    sig_tot = sig1+sig2+sig3

    display(Audio(data=sig_tot, rate=rate,autoplay=True, normalize=False))
    
    scale = 15
    spec1 = scale * get_spec_for_cm(wvl_1, a1)
    spec2 = scale * get_spec_for_cm(wvl_2, a2)
    spec3 = scale * get_spec_for_cm(wvl_3, a3)

    plt.figure(figsize=(4,4), facecolor='k')
    v = venn3(subsets=(1, 1, 1, 1, 1, 1, 1), set_labels = ('', '', ''))

    # iterate over each patch. Patches in matplotlib_venn are identified by
    # strings like '010', which is the intersection of the second set with the complements of 1st and 3rd.
    # I think.
    for elem in product([0,1],[0,1],[0,1]):
        if elem != (0,0,0):
            this_id = str(elem[0])+str(elem[1])+str(elem[2])  # eg '100', '010', etc
            # the color in the patch is the sum of the signal components represented in the patch
            this_spec = elem[0]*spec1 + elem[1]*spec2 + elem[2]*spec3 
            
            this_cmsc = cmsc_from_spec(this_spec)
            
            this_rgb = convert_color(this_cmsc, sRGBColor)
            this_clamped_rgb = sRGBColor(this_rgb.clamped_rgb_r,
                                         this_rgb.clamped_rgb_g,
                                         this_rgb.clamped_rgb_b)

            v.get_patch_by_id(this_id).set_color(this_clamped_rgb.get_rgb_hex())
            v.get_patch_by_id(this_id).set_alpha(1.0)
            v.get_label_by_id(this_id).set_text('')

    c = venn3_circles(subsets=(1, 1, 1, 1, 1, 1, 1), linestyle='-')

    plt.show()
    return

In [9]:
# autoplay interactive
v = interactive(tritone_metamers_cm, wvl_1=(381.0,779.0, 2), wvl_2=(381.0,779.0, 2), wvl_3=(381.0,779.0, 2), a1=(0.0,1.0, .05),
                a2=(0.0,1.0, .05), a3=(0.0,1.0, .05))
display(v)

interactive(children=(FloatSlider(value=460.0, description='wvl_1', max=779.0, min=381.0, step=2.0), FloatSlid…